<a href="https://colab.research.google.com/github/hwanys2/AI-math/blob/main/3.%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%9E%90%EB%A3%8C%EC%99%80%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5/3_1_%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

비정형화된 텍스트 자료를 처리하기 위해서는 전처리 과정들이 필요 합니다.  
2-2에서 뉴스 제목을 받아온 피클파일을 불러와서 데이터 처리 과정을 경험해 보겠습니다.   
혹시 앞 2-2실습을 통한 파일이 없으신 분은 다음 파일을 다운로드 하여 사용하시면 됩니다.  
[title_list 피클파일](https://drive.google.com/file/d/1FRlxMnPzDzRxxKvt41RxmrIT-w0lfjyI/view?usp=sharing)


# 파일 불러오기
우선 왼쪽에 폴더를 클릭합니다.  
title_list.pkl 파일을 끌어다 놓습니다. 그러면 업로드가 됩니다.  
이제 이 파일을 불러오겠습니다.  
다음과 같이 불러 옵니다.


In [ ]:
import pickle

with open('/content/title_list.pkl', 'rb') as f:
    title_list = pickle.load(f)
title_list

잘 가져와 진 것을 확인할 수 있습니다.  
이 자료를 가지고 맞춤법 수정, 중복문자 처리, 토큰화, 불용어 제거를 진행해 보겠습니다.
# 1. 맞춤법 검사
hanspell 라이브러리를 설치하고 이를 통해 맞춤법 검사를 수행해 보겠습니다.  
아래와 같이 설치하고 불러옵니다.

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

from hanspell import spell_checker

맞춤법 검사를 어떻게 되는지 한 번 진행해보겠습니다.

In [ ]:
sentence = '안냐하세요. 저는인공지능수학을 학습중인 학생입니댜.'
sentence_checked = spell_checker.check(sentence)
sentence_checked.checked

원본문장 : 안냐하세요. 저는인공지능수학을 학습중인 학생입니댜.
수정문장 : 안녕하세요. 저는 인공지능 수학을 학습 중인 학생입니다.
로 수정된 것을 확인할 수 있습니다.  
이를 이용하여 뉴스 제목들을 맞춤법 검사를 수행해 보겠습니다.  
뉴스 제목이라 맞춤법이 맞는 경우가 많아 크게 변화할 거 같진 않습니다만, 한 번 진행해보겠습니다. 시간이 다소 소요됩니다.

In [ ]:
spellchecked_title_list = []
i = 1
for title in title_list:
    if i % 20 == 0:
        print(f'{i}/{len(title_list)} 진행중 입니다. 잠시만 기다려 주세요.')
    i += 1
    try:
        spellchecked_title_list.append(spell_checker.check(title).checked)
    except:
        print('맞춤법 검사 실패 :', title)
spellchecked_title_list

# 2. 중복문자 제거하기
ㅋㅋㅋㅋㅋ나 ㅠㅠㅠㅠㅠ 와 같은 문자열을 많이 사용하곤 합니다. 이러한 문자열을 soynlp를 이용하여 통일 시켜주겠습니다.  
컴퓨터가 ㅋㅋㅋ나 ㅋㅋㅋㅋㅋㅋㅋ나 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 를 모두 다르게 인식한다면 같은 의미임에도 그 빈도수가 낮게 측정될 수 있기에 동일하게 만들어주기 위해 진행하는 작업이라고 보시면 되겠습니다.  
설치하고 어떻게 작동되는지 살펴보겠습니다.


In [ ]:
!pip install soynlp

from soynlp.normalizer import *

emoticon_nomalize와 repeat_nrmalize를 제공합니다.  
아래 코드를 실행시켜보시면 그 의미를 파악하실 수 있으실 겁니다.

In [ ]:
print(emoticon_normalize('앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ', num_repeats=2))

print(repeat_normalize('와하하하하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하핫', num_repeats=2))

뉴스제목들을 가져왔기에 중복문자 제거는 크게 의미가 있을거 같지 않아 생략합니다.  
# 3.토큰화
토큰화를 진행해보겠습니다.  
이를위해 코엔엘파이를 사용하겠습니다.  
코엔엘파이를 통해서 사용할 수 있는 형태소 분석기로 Okt(Open Korea Text), 메캅(Mecab), 코모란(Komoran), 한나눔(Hannanum), 꼬꼬마(Kkma)가 있습니다. 한국어 NLP에서 형태소 분석기를 사용한다는 것은 단어 토큰화가 아니라 정확히는 형태소(morpheme) 단위로 형태소 토큰화(morpheme tokenization)를 수행하게 됨을 뜻합니다. 여기선 이 Okt를 이용해보겠습니다.  
우선 설치 및 불러와보겠습니다.

In [ ]:
!pip install konlpy

from konlpy.tag import Okt
okt = Okt()

다음 문장을 형태소추출, 품사태깅, 명사추출 세가지로 진행해보겠습니다.  
> 인공지능 수학을 즐겁게 수업하는 선생님! 행복하세요. 



In [ ]:
# 형태소 추출
sentence = '인공지능 수학을 즐겁게 수업하는 선생님! 행복하세요.'
okt.morphs(sentence)

In [ ]:
# 품사태깅
okt.pos(sentence)

In [ ]:
# 명사추출
okt.nouns(sentence)

위와 같이 처리 됨을 확일 할 수 있습니다.  
품사태깅을 하여 뉴스 데이터를 담아보도록 하겠습니다. 

In [ ]:
pos_title_list = []
for title in title_list:
    pos_title_list.append(okt.pos(title))
pos_title_list

#4. 불용어 처리

텍스트 자료를 분석하기에 위 자료를 살펴보면 필요 없어 보이는 요소가 보입니다.  
자료 분석하는 사람의 선택의 문제일 수 있으나  
저는 조사나, 기호 등은 필요가 없어 보입니다.  
이를 제거하기 위해 먼저 okt에서 품사를 어떻게 태깅하는지 확인해보겠습니다.


In [ ]:
okt.tagset

이중에서 Josa, Number, PreEomi, Punctuation, Suffix, Unknown 을 제거해보겠습니다.

In [ ]:
preprocessed_title_list = []
for tokenized_title in pos_title_list:
    remove_stopword_tokenized_title = []
    for word, tag in tokenized_title:
        if tag not in ['Josa', 'Number', 'PreEomi','Punctuation', 'Suffix', 'Unknown',]:
            remove_stopword_tokenized_title.append((word, tag))
    preprocessed_title_list.append(remove_stopword_tokenized_title)
preprocessed_title_list

추가로 데이터를 보고 제거하고 싶은 불용어를 지정하여 제거할 수도 있습니다.  


이제 이 자료를 피클로 저장하고 후속 챕터로 이어가겠습니다.

In [ ]:
with open('/content/preprocessed_title_list.pkl', 'wb') as f:
    pickle.dump(preprocessed_title_list, f)

왼쪽 폴더를 누르면 preporcessed_title_list.pkl 파일이 생성된 것을 확인하실 수 있습니다.  
**이를 다운로드 받아 다음 챕터로 넘어가겠습니다.**